Creating new Master Peak file where peaks are divided into bins of 50 bp and the to total lenght of the peak is 5000 bp (2500 fome each side of the summit)

In [1]:
import pandas as pd 
import matplotlib as matplotlib
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
master_peak=pd.read_csv("master_peakCleaned_60nt.bed", sep="\t", header=None)
master_peak.columns = ["chr", "start", "end"]
master_peak.head()



,chr,start,end
0,chr1,3000354,3000621
1,chr1,3344239,3344375
2,chr1,3423800,3424266
3,chr1,3483016,3483312
4,chr1,3671770,3672445


In [3]:
#master_peak.shape
len(master_peak)
master_peak

,chr,start,end
0,chr1,3000354,3000621
1,chr1,3344239,3344375
2,chr1,3423800,3424266
3,chr1,3483016,3483312
4,chr1,3671770,3672445
...,...,...,...
24892,chrY,90808315,90808766
24893,chrY,90810161,90810301
24894,chrY,90810593,90811114
24895,chrY,90811889,90812674


In [4]:
### adding a new column showing the peak number#
for i in range(1,1+len(master_peak)):
    master_peak.loc[i-1, "peak_number"] = f"peak_{i}"#i
master_peak



,chr,start,end,peak_number
0,chr1,3000354,3000621,peak_1
1,chr1,3344239,3344375,peak_2
2,chr1,3423800,3424266,peak_3
3,chr1,3483016,3483312,peak_4
4,chr1,3671770,3672445,peak_5
...,...,...,...,...
24892,chrY,90808315,90808766,peak_24893
24893,chrY,90810161,90810301,peak_24894
24894,chrY,90810593,90811114,peak_24895
24895,chrY,90811889,90812674,peak_24896


In [7]:
master_peak["peak_mid"] = np.ceil((master_peak["start"] + master_peak["end"])/2)
master_peak["peak_length"] = np.ceil((master_peak["end"] - master_peak["start"])/2)*2
master_peak.head()

,chr,start,end,peak_number,peak_mid,peak_length
0,chr1,3000354,3000621,peak_1,3000488.0,268.0
1,chr1,3344239,3344375,peak_2,3344307.0,136.0
2,chr1,3423800,3424266,peak_3,3424033.0,466.0
3,chr1,3483016,3483312,peak_4,3483164.0,296.0
4,chr1,3671770,3672445,peak_5,3672108.0,676.0


In [24]:
### creating new dataframe with bin size of 50 and extetend length of peak by 2500bp on each side of the summit ###

#master_peak_bind=pd.DataFrame(columns=["chr", "start", "end", "peak_number", "peak_length"])

columns=["chr", "start", "end", "peak_number", "peak_length"]

master_peak_bin=pd.DataFrame(0, index=range(len(master_peak)*100),columns=columns)
master_peak_bin
#df = pd.DataFrame(0, index=range(24897), columns=columns)


,chr,start,end,peak_number,peak_length
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2489695,0,0,0,0,0
2489696,0,0,0,0,0
2489697,0,0,0,0,0
2489698,0,0,0,0,0


In [11]:
rows = []
for i in range(1, 1 + len(master_peak)):
    for j in range(-2500, 2500, 50):
        rows.append({"chr": master_peak.loc[i-1, "chr"],
                     "start": master_peak.loc[i-1, "peak_mid"] + j - 25,
                     "end": master_peak.loc[i-1, "peak_mid"] + j + 25,
                     "peak_number": master_peak.loc[i-1, "peak_number"],
                     "peak_length": 50})

# Convert list to DataFrame once
master_peak_bind = pd.DataFrame(rows)


In [16]:
master_peak_bind

,chr,start,end,peak_number,peak_length
0,chr1,2997963.0,2998013.0,peak_1,50
1,chr1,2998013.0,2998063.0,peak_1,50
2,chr1,2998063.0,2998113.0,peak_1,50
3,chr1,2998113.0,2998163.0,peak_1,50
4,chr1,2998163.0,2998213.0,peak_1,50
...,...,...,...,...,...
2489695,chrY,90821007.0,90821057.0,peak_24897,50
2489696,chrY,90821057.0,90821107.0,peak_24897,50
2489697,chrY,90821107.0,90821157.0,peak_24897,50
2489698,chrY,90821157.0,90821207.0,peak_24897,50


In [32]:
# make master_peak_bind[["start", "end"]] as type int
master_peak_bind[["start", "end"]] = master_peak_bind[["start", "end"]].astype(int)

# remove chrUn_GL456370 and chrM
master_peak_bind = master_peak_bind[master_peak_bind["chr"] != "chrUn_GL456370"]
master_peak_bind = master_peak_bind[master_peak_bind["chr"] != "chrM"]

# save master_peak_bind to a tab separated file without the index and  header
master_peak_bind[["chr", "start", "end"]].to_csv("master_peakCleaned_60nt_binned.bed", sep="\t", index=False, header=False)


In [33]:
# test if start or end is negative
master_peak_bind[master_peak_bind["start"] < 0]['chr'].value_counts()


Series([], Name: count, dtype: int64)